# Automated Brain Extraction
In this code we use recon-all funtion for brain extraction.The recon-all command from FreeSurfer software is used to process MRI scans, performing automated brain extraction and cortical reconstruction.


-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [1]:
%%bash
# Define SUBJECTS_DIR explicitly
SUBJECTS_DIR="/mnt/d/Data/Human/ObjectCategorization/analysis2"

# Ensure Freesurfer directory exists
mkdir -p "$SUBJECTS_DIR/BrainExtraction"

# List relevant T1w files from session 01, extract subject IDs, and process in parallel
ls "$SUBJECTS_DIR"/sub*/ses-01/anat/*T1w.nii.gz -1 | awk -F'/' '{print $8}' | \
parallel --jobs 4 recon-all -s "$SUBJECS_DIR/BrainExtraction/anat_ses-01_{}" \
-i "$SUBJECTS_DIR/{}/ses-01/anat/{}*_ses-01_T1w.nii.gz" -all -qcache -3T

# List relevant T1w filesfrom sesion 02, extract subject IDs, and process in parallel
ls "$SUBJECTS_DIR"/sub*/ses-02/anat/*T1w.nii.gz -1 | awk -F'/' '{print $8}' | \
parallel --jobs 4 recon-all -s "$SUBJECTS_DIR/BrainExtraction/anat_ses-02_{}" \
-i "$SUBJECTS_DIR/{}/ses-02/anat/{}*_ses-02_T1w.nii.gz" -all -qcache -3T


#move results to the Freesurfer folder
mv "$SUBJECTS_DIR"/anat_ses-0* "$SUBJECTS_DIR/BrainExtraction/"

Process is terminated.


# Conver Brain Masks to nifti format
In addition to using recon-all from FreeSurfer for brain reconstruction, we also applied BET from the FSL package to enhance brain extraction, ensuring greater clarity and accuracy in the results.

In [10]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find anatomical directories
anat_dirs = list(data_path.glob("BrainExtraction/anat*/mri"))

# Process each anatomy directory
for curr_anat in anat_dirs:
    print(f"Change Directory to >> {curr_anat}")
    

    # Ensure the directory exists before changing into it
    if not curr_anat.exists():
        print(f"Skipping: Directory {curr_anat} does not exist!")
        continue
    
    os.chdir(curr_anat)
    
    # Convert brain.mgz to brain.nii.gz
    print("\t1) Converting brain.mgz to brain.nii.gz.")
    os.system(f"mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz")

    # Run BET function from FSL package to get a clear brain mask
    print("\t2) Running BET function from FSL package...")
    os.system("bet brain brain_brain -R -f 0.3 -g 0 -m")

    # Define output path dynamically
    subject = curr_anat.parts[-2].split("_")[-1]
    session = curr_anat.parts[-2].split("_")[-2]
    out_put = data_path / subject / session / "anat" / "mask.nii.gz"

    print(f"\t3) Copying brain mask to {out_put.parent}")
    os.system(f"cp brain_brain_mask.nii.gz {out_put}")

    # Apply brain mask on the original image
    print("\t4) Applying brain mask on original image")
    t1w_image = list(out_put.parent.glob("*T1w.nii.gz"))
    out_put_brain = data_path / subject / session / "anat" / "brain.nii.gz"

    if t1w_image:
        os.system(f"fslmaths {t1w_image[0]} -mul {out_put} {out_put_brain}")
    else:
        print(f"Skipping: No T1w.nii.gz file found in {out_put.parent}")

    # Deleting the nifti files from Freesurfer folders
    os.system('rm -r *nii.gz')
    print("\n")


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-01/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.46, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 3.72529e-09)
j_ras = (0, 2.98023e-08, -1)
k_ras = (-1.86265e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-01/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-02/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.33, TE=0.00, TI=0.00, 

reading from brain.mgz...
TR=8.36, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -9.31323e-10, 0)
j_ras = (0, 0, -1)
k_ras = (-9.31323e-10, 1, -2.98023e-08)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-12/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-13/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.50, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -1.62981e-08, -1.86265e-09)
j_ras = (1.86265e-09, 2.98023e-08, -1)
k_ras = (1.39698e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (n

	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-23/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-24/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.32, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -4.65661e-10, -3.72529e-09)
j_ras = (-7.45058e-09, -1.49012e-08, -1)
k_ras = (-2.32831e-09, 1, 1.49012e-08)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-24/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-0



Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-35/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.17, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -4.48199e-09, 0)
j_ras = (1.86265e-09, 0, -1)
k_ras = (4.48199e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-35/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-36/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.25, TE=0.00, TI=0.00

reading from brain.mgz...
TR=8.19, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -3.72529e-09, 1.86265e-09)
j_ras = (-1.86265e-09, 0, -1)
k_ras = (0, 1, 3.72529e-09)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-46/ses-01/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-01_sub-47/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.47, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -9.31323e-10, -3.72529e-09)
j_ras = (3.72529e-09, 0, -1)
k_ras = (9.31323e-10, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (no

...

reading from brain.mgz...
TR=8.22, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -1.62981e-09, 1.86265e-09)
j_ras = (1.86265e-09, 0, -1)
k_ras = (-4.65661e-10, 1, -7.45058e-09)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-30/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-31/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.35, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (-7.45058e-09, 0, -1)
k_ras = (-1.86265e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0

	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-41/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-42/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.18, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -3.49246e-10, -1.86265e-09)
j_ras = (0, 0, -1)
k_ras = (3.49246e-10, 1, 7.45058e-09)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-42/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-43/mri
	1) Conver



Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-53/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.25, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, -3.14321e-09, 4.65661e-10)
j_ras = (-4.65661e-10, 0, -1)
k_ras = (3.14321e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-53/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-54/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.20, TE=0.

reading from brain.mgz...
TR=8.54, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (2.98023e-08, -2.98023e-08, -1)
k_ras = (-7.45058e-09, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-64/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-65/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.17, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 9.31323e-10, 0)
j_ras = (7.45058e-09, 0, -1)
k_ras = (-9.31323e-10, 1, 1.86265e-09)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0).

	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-75/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-76/mri
	1) Converting brain.mgz to brain.nii.gz.
mri_convert --reslice_like orig/001.mgz brain.mgz brain.nii.gz 
reading from brain.mgz...
TR=8.23, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 3.20142e-10, 0)
j_ras = (2.32831e-10, 7.45058e-09, -1)
k_ras = (6.1118e-10, 1, 0)
reading template info from volume orig/001.mgz...
changing data type from uchar to float (noscale = 0)...
Reslicing using trilinear interpolation 
writing to brain.nii.gz...
	2) Running BET function from FSL package...
	3) Copying brain mask to /mnt/d/Data/Human/ObjectCategorization/analysis2/sub-76/ses-02/anat
	4) Applying brain mask on original image


Change Directory to >> /mnt/d/Data/Human/ObjectCategorization/analysis2/BrainExtraction/anat_ses-02_sub-77/mri
	1) Convertin